# --- Day 8: Playground ---
https://adventofcode.com/2025/day/8

In my [first crack at this problem](https://github.com/kevinxperese/advent-of-code/blob/3851eb1dca84d1aea829af566bab014cbf62a86b/2025/notebooks/day-08.ipynb), I solved Part 1 without really knowing what I was doing.

I started off with reading in all the 3D coordinates for each box as `numpy` arrays, so I could "easily"/efficiently(?) calculate the Euclidean distances between each of them using `np.linalg.norm()`.

I knew that I would use `itertools.combinations()` to get all the possible pairs between the boxes and I had what I thought was a pretty slow approach by sorting them all by their distances. I then turned to `scipy.spacial.distances.pdist()` to measure all the distances and then used `numpy.triu_indices` to map those back to pairs of boxes, **and** used a prioirty queue (`heapq`) to skip the sorting. This all felt like too much.

Once I had those pairs sorted by distances, my `make_circuts()` function was where the real problem was. I was using `itertools.combinations()` again to create new circuts in place. I did piece together on my own that I needed to use `set()` objects and test for intersections and create unions to build up the circuts, like the problem specified.

While I was able to solve Part 1 with this approach, I wasn't able to figure out how to get a solution to Part 2.

So I spent some time (days) thinking about this and reading about approaches.

## Key Insight
The key insight to solving this problem efficiently is realizing that it's a *graph* problem.

Specifically: Each box is a *node*/*vertice* and the distance between each is the *weight* for each *edge*. As such, the graph is *fully connected.* (Each node is connect to all the other nodes.)

Cool, cool.

The *goal* for solving the problem is to create a [minimum spanning tree](https://en.wikipedia.org/wiki/Minimum_spanning_tree) from the graph.

There are several algorithms for accomplishing this. Two popular ones are:
* [Prim's algorithm](https://en.wikipedia.org/wiki/Prim%27s_algorithm)
* [Kruskal's algorithm](https://en.wikipedia.org/wiki/Kruskal%27s_algorithm), and

Here's a key phrase from the Wikipedia description of Kruskal's algorithm:
> It's a greedy algorighm that in each step adds to the forest the lowest-weight edge that will not form a cycle.

This lines up with these sentences from the problem:
> The next two junction boxes are 431,825,988 and 425,690,689. Because these two junction boxes were already in the same circuit, nothing happens!

If an edge with two nodes that are already in a circut are added to that "component"/"circut", then that would introduce a cycle

## Parse the Input Data

In [1]:
def parse(filename):
    """Parse puzzle input data."""
    with open(f'../inputs/{filename}.txt') as f:
        return [tuple(map(int, line.split(','))) for line in f]  # junction box coordinates

In [2]:
parse("day-08_test")

[(162, 817, 812),
 (57, 618, 57),
 (906, 360, 560),
 (592, 479, 940),
 (352, 342, 300),
 (466, 668, 158),
 (542, 29, 236),
 (431, 825, 988),
 (739, 650, 466),
 (52, 470, 668),
 (216, 146, 977),
 (819, 987, 18),
 (117, 168, 530),
 (805, 96, 715),
 (346, 949, 466),
 (970, 615, 88),
 (941, 993, 340),
 (862, 61, 35),
 (984, 92, 344),
 (425, 690, 689)]

## Part 1
---

In [3]:
import heapq
from itertools import combinations
from math import prod

import numpy as np
from scipy.spatial.distance import pdist  # pairwise distances

In [4]:
# No longer used
def euc_dist(b1, b2):
    """Euclidean distance, sans square root calc, which isn't necessary."""
    return sum([pow(x - y, 2) for x, y in zip(b1, b2)])

In [5]:
def get_sorted_pairs(boxes):
    """Sort all combinations of boxes by Euclidean distance."""

    # One-liner, but kinda slow...
    # pairs = sorted(combinations(boxes, 2), key=lambda pair: euc_dist(*pair))

    pq = []  # priority queue

    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html
    # https://numpy.org/doc/stable/reference/generated/numpy.triu_indices.html
    distances = pdist(np.array(boxes))
    indicies = zip(*np.triu_indices(len(boxes), k=1))

    for d, (i, j) in zip(distances, indicies):
        heapq.heappush(pq, (d, (boxes[i], boxes[j])))

    return pq

In [6]:
def connect(circuts, sorted_pairs, n):
    """Kruskal's algorithm-ish."""
    i = 0

    while sorted_pairs:
        _, (b1, b2) = heapq.heappop(sorted_pairs)
        i += 1
        c_with_b1 = [c for c in circuts if b1 in c][0]
        c_with_b2 = [c for c in circuts if b2 in c][0]

        if c_with_b1 != c_with_b2:
            c_with_b1.update(c_with_b2)
            c_with_b2.clear()

        if i == n:
            return circuts

In [7]:
def solve(boxes, n, t):
    circuts = [{b} for b in boxes]  # Each node (box) starts off as set()
    sorted_pairs = get_sorted_pairs(boxes)  # All possible pairs of boxes, sorted by distance
    msf = connect(circuts, sorted_pairs, n)  # minimum spanning forest
    return prod(sorted([len(t) for t in msf])[-1*t:])  # product of the size of the t largest trees

### Run on Test Data

In [8]:
solve(parse("day-08_test"), 10, 3) == 40

True

### Run on Input Data

In [9]:
solve(parse("day-08"), 1000, 3)

127551

## Part 2
---

In [10]:
def connect2(circuts, sorted_pairs):
    n = len(circuts)
    i = 0

    while sorted_pairs:
        _, (b1, b2) = heapq.heappop(sorted_pairs)
        c_with_b1 = [c for c in circuts if b1 in c][0]
        c_with_b2 = [c for c in circuts if b2 in c][0]

        if c_with_b1 != c_with_b2:
            c_with_b1.update(c_with_b2)
            c_with_b2.clear()
            i += 1

        if i == n - 1:
            return b1, b2

In [11]:
def solve2(boxes):
    circuts = [{b} for b in boxes]  # Each node (box) starts off as set()
    sorted_pairs = get_sorted_pairs(boxes)  # All possible pairs of boxes, sorted by distance
    last_connection = connect2(circuts, sorted_pairs)
    b1, b2 = last_connection
    return b1[0] * b2[0]

### Run on Test Data

In [12]:
solve2(parse("day-08_test")) == 25272

True

### Run on Input Data

In [13]:
solve2(parse("day-08"))

2347225200